# Using MNE package for data storage.
## inspired from https://www.kaggle.com/code/nizarislah/sleep-apnea-notebook

In [ ]:
import sys
from os import listdir
from os.path import isfile, join
from datetime import  timezone

from tqdm.notebook import tqdm
import mne



from pyapnea.oscar.oscar_loader import load_session
from pyapnea.oscar.oscar_getter import event_data_to_dataframe, get_channel_from_code
from pyapnea.oscar.oscar_constants import CHANNELS, ChannelID

In [ ]:
# all files
data_path_cpap1 = '../data/raw/ResMed_23192565579/Events'
list_files = [{'label': f, 'value': f, 'fullpath': join(data_path_cpap1, f)} for f in listdir(data_path_cpap1) if isfile(join(data_path_cpap1, f))]
data_path_cpap2= '../data/raw/ResMed_23221085377/Events'
list_files.extend([{'label': f, 'value': f, 'fullpath': join(data_path_cpap2, f)} for f in listdir(data_path_cpap2) if isfile(join(data_path_cpap2, f))])


In [ ]:
filename_to_load = '../data/raw/ResMed_23192565579/Events/62202198.001'
oscar_session_data = load_session(filename_to_load)
df = event_data_to_dataframe(oscar_session_data, ChannelID.CPAP_FlowRate.value)

sfreq = 25

df

In [ ]:
info = mne.create_info(ch_names=['FlowRate'] * 1 ,
                       ch_types=['misc'] * 1,
                       sfreq=sfreq)
#print(df['time_utc'].iloc[0].to_pydatetime())
raw = mne.io.RawArray(df[['FlowRate']].transpose(), info)
d = df['time_utc'].iloc[0].to_pydatetime().astimezone(timezone.utc)
raw.set_meas_date(d)
raw.plot(show_scrollbars=False, show_scalebars=False,
         time_format='clock',
         scalings=dict(misc=50))

In [ ]:
#loading all files into mne structures
raw_array = []
df_array = []
with tqdm(total=len(list_files)) as pbar:
    for f in list_files:
        oscar_session_data = load_session(f['fullpath'])
        df = event_data_to_dataframe(oscar_session_data, ChannelID.CPAP_FlowRate.value)
        info = mne.create_info(ch_names=['FlowRate'] * 1 ,
                       ch_types=['misc'] * 1,
                       sfreq=sfreq)
        raw = mne.io.RawArray(df[['FlowRate']].transpose(), info, verbose=False)
        d = df['time_utc'].iloc[0].to_pydatetime().astimezone(timezone.utc)
        raw.set_meas_date(d)
        raw_array.append(raw)
        pbar.update(1)
